In [1]:
# Generic inputs for most ML tasks
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn import tree
import graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# setup interactive notebook mode
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display, HTML

/var/folders/tl/lnf2sv191t77b2f4hhxqlcx80000gn/T/ipykernel_80757/27391427.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Predictions for April 9th, 2024: 

In [44]:
initial_predictions_dataset = pd.read_csv("./test_data/CIS_662 _INITIAL_Predictions.csv")

In [45]:
initial_predictions_dataset.head(10)

,DATE,DAY,FLIGHT NUMBER,MKT_UNIQUE_CARRIER,OP_UNIQUE_CARRIER,ORIGIN,DEPARTURE TIME,ARRIVAL TIME,ARRIVAL STATUS,ARRIVAL STATUS_Prev_flight_early,ARRIVAL STATUS_Prev_flight_ontime,ARRIVAL STATUS_Prev_flight_late
0,4/10/24,WEDNESDAY,UA 1400,UA,UA,ORD,6:52 PM,9:47 PM,NaN,NaN,NaN,NaN
1,4/10/24,WEDNESDAY,AA 3402,AA,MQ,ORD,7:59 PM,10:52 PM,NaN,NaN,NaN,NaN
2,4/10/24,WEDNESDAY,B6 116,B6,B6,JFK,1:33 PM,2:50 PM,NaN,NaN,NaN,NaN
3,4/10/24,WEDNESDAY,DL 5182,DL,9E,JFK,2:55 PM,4:21 PM,NaN,NaN,NaN,NaN
4,4/10/24,WEDNESDAY,WN 5285,WN,WN,MCO,11:05 AM,1:45 PM,NaN,NaN,NaN,NaN
5,4/10/24,WEDNESDAY,B6 656,B6,B6,MCO,1:35 PM,4:25 PM,NaN,NaN,NaN,NaN
6,4/11/24,THURSDAY,UA 1400,UA,UA,ORD,6:52 PM,9:47 PM,NaN,NaN,NaN,NaN
7,4/11/24,THURSDAY,AA 3402,AA,MQ,ORD,7:59 PM,10:52 PM,NaN,NaN,NaN,NaN
8,4/11/24,THURSDAY,B6 116,B6,B6,JFK,1:33 PM,2:50 PM,NaN,NaN,NaN,NaN
9,4/11/24,THURSDAY,DL 5182,DL,9E,JFK,2:55 PM,4:21 PM,NaN,NaN,NaN,NaN


In [46]:
initial_predictions_dataset.columns

Index(['DATE', 'DAY', 'FLIGHT NUMBER', 'MKT_UNIQUE_CARRIER',
       'OP_UNIQUE_CARRIER', 'ORIGIN', 'DEPARTURE TIME', 'ARRIVAL TIME',
       'ARRIVAL STATUS', 'ARRIVAL STATUS_Prev_flight_early',
       'ARRIVAL STATUS_Prev_flight_ontime', 'ARRIVAL STATUS_Prev_flight_late'],
      dtype='object')

In [47]:
initial_predictions_dataset.dtypes

DATE                                  object
DAY                                   object
FLIGHT NUMBER                         object
MKT_UNIQUE_CARRIER                    object
OP_UNIQUE_CARRIER                     object
ORIGIN                                object
DEPARTURE TIME                        object
ARRIVAL TIME                          object
ARRIVAL STATUS                       float64
ARRIVAL STATUS_Prev_flight_early     float64
ARRIVAL STATUS_Prev_flight_ontime    float64
ARRIVAL STATUS_Prev_flight_late      float64
dtype: object

In [48]:
def convert_to_24hr_format(date, time):
    # Adjust the format to match the 'month/day/year' structure
    return pd.to_datetime(date + ' ' + time, format='%m/%d/%y %I:%M %p')

initial_predictions_dataset['SCH_DEP_TIME'] = initial_predictions_dataset.apply(
    lambda row: convert_to_24hr_format(row['DATE'], row['DEPARTURE TIME']), axis=1)
initial_predictions_dataset['SCH_ARR_TIME'] = initial_predictions_dataset.apply(
    lambda row: convert_to_24hr_format(row['DATE'], row['ARRIVAL TIME']), axis=1)

initial_predictions_dataset.head(10)

,DATE,DAY,FLIGHT NUMBER,MKT_UNIQUE_CARRIER,OP_UNIQUE_CARRIER,ORIGIN,DEPARTURE TIME,ARRIVAL TIME,ARRIVAL STATUS,ARRIVAL STATUS_Prev_flight_early,ARRIVAL STATUS_Prev_flight_ontime,ARRIVAL STATUS_Prev_flight_late,SCH_DEP_TIME,SCH_ARR_TIME
0,4/10/24,WEDNESDAY,UA 1400,UA,UA,ORD,6:52 PM,9:47 PM,NaN,NaN,NaN,NaN,2024-04-10 18:52:00,2024-04-10 21:47:00
1,4/10/24,WEDNESDAY,AA 3402,AA,MQ,ORD,7:59 PM,10:52 PM,NaN,NaN,NaN,NaN,2024-04-10 19:59:00,2024-04-10 22:52:00
2,4/10/24,WEDNESDAY,B6 116,B6,B6,JFK,1:33 PM,2:50 PM,NaN,NaN,NaN,NaN,2024-04-10 13:33:00,2024-04-10 14:50:00
3,4/10/24,WEDNESDAY,DL 5182,DL,9E,JFK,2:55 PM,4:21 PM,NaN,NaN,NaN,NaN,2024-04-10 14:55:00,2024-04-10 16:21:00
4,4/10/24,WEDNESDAY,WN 5285,WN,WN,MCO,11:05 AM,1:45 PM,NaN,NaN,NaN,NaN,2024-04-10 11:05:00,2024-04-10 13:45:00
5,4/10/24,WEDNESDAY,B6 656,B6,B6,MCO,1:35 PM,4:25 PM,NaN,NaN,NaN,NaN,2024-04-10 13:35:00,2024-04-10 16:25:00
6,4/11/24,THURSDAY,UA 1400,UA,UA,ORD,6:52 PM,9:47 PM,NaN,NaN,NaN,NaN,2024-04-11 18:52:00,2024-04-11 21:47:00
7,4/11/24,THURSDAY,AA 3402,AA,MQ,ORD,7:59 PM,10:52 PM,NaN,NaN,NaN,NaN,2024-04-11 19:59:00,2024-04-11 22:52:00
8,4/11/24,THURSDAY,B6 116,B6,B6,JFK,1:33 PM,2:50 PM,NaN,NaN,NaN,NaN,2024-04-11 13:33:00,2024-04-11 14:50:00
9,4/11/24,THURSDAY,DL 5182,DL,9E,JFK,2:55 PM,4:21 PM,NaN,NaN,NaN,NaN,2024-04-11 14:55:00,2024-04-11 16:21:00


In [49]:
initial_predictions_dataset.drop(columns=['DATE', 'DEPARTURE TIME', 'ARRIVAL TIME'], inplace = True) 
initial_predictions_dataset.head()

,DAY,FLIGHT NUMBER,MKT_UNIQUE_CARRIER,OP_UNIQUE_CARRIER,ORIGIN,ARRIVAL STATUS,ARRIVAL STATUS_Prev_flight_early,ARRIVAL STATUS_Prev_flight_ontime,ARRIVAL STATUS_Prev_flight_late,SCH_DEP_TIME,SCH_ARR_TIME
0,WEDNESDAY,UA 1400,UA,UA,ORD,NaN,NaN,NaN,NaN,2024-04-10 18:52:00,2024-04-10 21:47:00
1,WEDNESDAY,AA 3402,AA,MQ,ORD,NaN,NaN,NaN,NaN,2024-04-10 19:59:00,2024-04-10 22:52:00
2,WEDNESDAY,B6 116,B6,B6,JFK,NaN,NaN,NaN,NaN,2024-04-10 13:33:00,2024-04-10 14:50:00
3,WEDNESDAY,DL 5182,DL,9E,JFK,NaN,NaN,NaN,NaN,2024-04-10 14:55:00,2024-04-10 16:21:00
4,WEDNESDAY,WN 5285,WN,WN,MCO,NaN,NaN,NaN,NaN,2024-04-10 11:05:00,2024-04-10 13:45:00


In [50]:
# Define a mapping from day names to numbers
day_to_number = {
    'MONDAY': 1,
    'TUESDAY': 2,
    'WEDNESDAY': 3,
    'THURSDAY': 4,
    'FRIDAY': 5,
    'SATURDAY': 6,
    'SUNDAY': 7
}

# Apply the mapping to the 'DAY' column to create 'DAY_OF_WEEK'
initial_predictions_dataset['DAY_OF_WEEK'] = initial_predictions_dataset['DAY'].map(day_to_number)
initial_predictions_dataset.drop(columns=['DAY'], inplace=True)
# Verify the changes
initial_predictions_dataset.head()


,FLIGHT NUMBER,MKT_UNIQUE_CARRIER,OP_UNIQUE_CARRIER,ORIGIN,ARRIVAL STATUS,ARRIVAL STATUS_Prev_flight_early,ARRIVAL STATUS_Prev_flight_ontime,ARRIVAL STATUS_Prev_flight_late,SCH_DEP_TIME,SCH_ARR_TIME,DAY_OF_WEEK
0,UA 1400,UA,UA,ORD,NaN,NaN,NaN,NaN,2024-04-10 18:52:00,2024-04-10 21:47:00,3
1,AA 3402,AA,MQ,ORD,NaN,NaN,NaN,NaN,2024-04-10 19:59:00,2024-04-10 22:52:00,3
2,B6 116,B6,B6,JFK,NaN,NaN,NaN,NaN,2024-04-10 13:33:00,2024-04-10 14:50:00,3
3,DL 5182,DL,9E,JFK,NaN,NaN,NaN,NaN,2024-04-10 14:55:00,2024-04-10 16:21:00,3
4,WN 5285,WN,WN,MCO,NaN,NaN,NaN,NaN,2024-04-10 11:05:00,2024-04-10 13:45:00,3


In [51]:

# Negative "diff_arrival_departure" indicates that flight arrive next day.
diff_arrival_departure = initial_predictions_dataset['SCH_ARR_TIME'] - initial_predictions_dataset['SCH_DEP_TIME']


initial_predictions_dataset = initial_predictions_dataset[diff_arrival_departure > pd.Timedelta(0)]

initial_predictions_dataset.shape
initial_predictions_dataset.dtypes

(23, 11)

FLIGHT NUMBER                                object
MKT_UNIQUE_CARRIER                           object
OP_UNIQUE_CARRIER                            object
ORIGIN                                       object
ARRIVAL STATUS                              float64
ARRIVAL STATUS_Prev_flight_early            float64
ARRIVAL STATUS_Prev_flight_ontime           float64
ARRIVAL STATUS_Prev_flight_late             float64
SCH_DEP_TIME                         datetime64[ns]
SCH_ARR_TIME                         datetime64[ns]
DAY_OF_WEEK                                   int64
dtype: object

In [52]:
initial_predictions_dataset['DEST'] = 'SYR'

In [53]:
weather_data = pd.read_csv('./test_data/initial_test_weather.csv', parse_dates=['datetime'])
weather_data.head()
weather_data.shape
weather_data.dtypes

,name,datetime,temp,feelslike,dew,humidity,precip,precipprob,preciptype,snow,snowdepth,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations
0,new york,2024-04-08 00:00:00,46.30,45.10,35.00,64.60,0.00,0.00,NaN,0,0.00,8.10,3.40,267.00,"1,021.40",0.00,9.90,0,0.00,0,10,Clear,clear-night,"KLGA,KJRB,F1417,KNYC"
1,new york,2024-04-08 01:00:00,46.20,42.80,35.00,64.86,0.00,0.00,NaN,0,0.00,8.10,6.70,262.00,"1,021.60",0.00,9.90,0,0.00,0,10,Clear,clear-night,"KLGA,KJRB,F1417,KNYC"
2,new york,2024-04-08 02:00:00,46.20,43.20,35.10,64.91,0.00,0.00,NaN,0,0.00,8.10,5.90,262.00,"1,021.50",0.00,9.90,0,0.00,0,10,Clear,clear-night,"KLGA,KJRB,F1417,KNYC"
3,new york,2024-04-08 03:00:00,45.20,42.10,34.90,67.12,0.00,0.00,NaN,0,0.00,8.10,5.70,262.00,"1,021.50",0.00,9.90,0,0.00,0,10,Clear,clear-night,"KLGA,KJRB,F1417,KNYC"
4,new york,2024-04-08 04:00:00,45.10,42.20,35.70,69.33,0.00,0.00,NaN,0,0.00,6.90,5.50,285.00,"1,021.40",0.00,9.90,0,0.00,0,10,Clear,clear-night,"KLGA,KJRB,F1417,KNYC"


(1152, 24)

name                        object
datetime            datetime64[ns]
temp                       float64
feelslike                  float64
dew                        float64
humidity                   float64
precip                     float64
precipprob                 float64
preciptype                  object
snow                         int64
snowdepth                  float64
windgust                   float64
windspeed                  float64
winddir                    float64
sealevelpressure           float64
cloudcover                 float64
visibility                 float64
solarradiation               int64
solarenergy                float64
uvindex                      int64
severerisk                   int64
conditions                  object
icon                        object
stations                    object
dtype: object

In [54]:
print(set(weather_data['name']))

{'new york', 'chicago', 'orlando', 'syracuse'}


In [55]:
# Replacing city names with airport names
dic = {'syracuse': 'SYR', 'new york': 'JFK', 'orlando': 'MCO', 'chicago': 'ORD'}

weather_data['name'] = weather_data['name'].apply(lambda row: dic[row.lower()])

weather_data.head()
print(set(weather_data['name']))

,name,datetime,temp,feelslike,dew,humidity,precip,precipprob,preciptype,snow,snowdepth,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations
0,JFK,2024-04-08 00:00:00,46.30,45.10,35.00,64.60,0.00,0.00,NaN,0,0.00,8.10,3.40,267.00,"1,021.40",0.00,9.90,0,0.00,0,10,Clear,clear-night,"KLGA,KJRB,F1417,KNYC"
1,JFK,2024-04-08 01:00:00,46.20,42.80,35.00,64.86,0.00,0.00,NaN,0,0.00,8.10,6.70,262.00,"1,021.60",0.00,9.90,0,0.00,0,10,Clear,clear-night,"KLGA,KJRB,F1417,KNYC"
2,JFK,2024-04-08 02:00:00,46.20,43.20,35.10,64.91,0.00,0.00,NaN,0,0.00,8.10,5.90,262.00,"1,021.50",0.00,9.90,0,0.00,0,10,Clear,clear-night,"KLGA,KJRB,F1417,KNYC"
3,JFK,2024-04-08 03:00:00,45.20,42.10,34.90,67.12,0.00,0.00,NaN,0,0.00,8.10,5.70,262.00,"1,021.50",0.00,9.90,0,0.00,0,10,Clear,clear-night,"KLGA,KJRB,F1417,KNYC"
4,JFK,2024-04-08 04:00:00,45.10,42.20,35.70,69.33,0.00,0.00,NaN,0,0.00,6.90,5.50,285.00,"1,021.40",0.00,9.90,0,0.00,0,10,Clear,clear-night,"KLGA,KJRB,F1417,KNYC"


{'ORD', 'SYR', 'MCO', 'JFK'}


In [56]:
weather_columns_to_drop = ['dew','humidity','snowdepth','sealevelpressure', 'solarradiation', 'solarenergy','uvindex','icon','stations']
weather_data.drop(columns = weather_columns_to_drop, inplace=True)

In [57]:
# Create two data frame for origin and destination. Also rename the columns by adding the prefix.
rename_origin = {}
rename_dest = {}
for col in weather_data.columns:
    rename_origin[col] = 'ORGIN_WTH_' + col
    rename_dest[col] = 'DEST_WTH_' + col
    
org_weather_data = weather_data.rename(columns=rename_origin)
dst_weather_data = weather_data.rename(columns=rename_dest)

In [58]:
# Adding Join columns
initial_predictions_dataset['ORGIN_WTH_JOIN'] = initial_predictions_dataset['SCH_DEP_TIME'].dt.round('H').astype(str) + initial_predictions_dataset['ORIGIN']
initial_predictions_dataset['DEST_WTH_JOIN'] = initial_predictions_dataset['SCH_ARR_TIME'].dt.round('H').astype(str) + initial_predictions_dataset['DEST']
org_weather_data['ORGIN_WTH_JOIN'] = org_weather_data['ORGIN_WTH_datetime'].astype(str) + org_weather_data['ORGIN_WTH_name']
dst_weather_data['DEST_WTH_JOIN'] = dst_weather_data['DEST_WTH_datetime'].astype(str) + dst_weather_data['DEST_WTH_name']
initial_predictions_dataset.head()
org_weather_data.head()
dst_weather_data.head()

/var/folders/tl/lnf2sv191t77b2f4hhxqlcx80000gn/T/ipykernel_80757/2442795551.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  initial_predictions_dataset['ORGIN_WTH_JOIN'] = initial_predictions_dataset['SCH_DEP_TIME'].dt.round('H').astype(str) + initial_predictions_dataset['ORIGIN']
/var/folders/tl/lnf2sv191t77b2f4hhxqlcx80000gn/T/ipykernel_80757/2442795551.py:3: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  initial_predictions_dataset['DEST_WTH_JOIN'] = initial_predictions_dataset['SCH_ARR_TIME'].dt.round('H').astype(str) + initial_predictions_dataset['DEST']


,FLIGHT NUMBER,MKT_UNIQUE_CARRIER,OP_UNIQUE_CARRIER,ORIGIN,ARRIVAL STATUS,ARRIVAL STATUS_Prev_flight_early,ARRIVAL STATUS_Prev_flight_ontime,ARRIVAL STATUS_Prev_flight_late,SCH_DEP_TIME,SCH_ARR_TIME,DAY_OF_WEEK,DEST,ORGIN_WTH_JOIN,DEST_WTH_JOIN
0,UA 1400,UA,UA,ORD,NaN,NaN,NaN,NaN,2024-04-10 18:52:00,2024-04-10 21:47:00,3,SYR,2024-04-10 19:00:00ORD,2024-04-10 22:00:00SYR
1,AA 3402,AA,MQ,ORD,NaN,NaN,NaN,NaN,2024-04-10 19:59:00,2024-04-10 22:52:00,3,SYR,2024-04-10 20:00:00ORD,2024-04-10 23:00:00SYR
2,B6 116,B6,B6,JFK,NaN,NaN,NaN,NaN,2024-04-10 13:33:00,2024-04-10 14:50:00,3,SYR,2024-04-10 14:00:00JFK,2024-04-10 15:00:00SYR
3,DL 5182,DL,9E,JFK,NaN,NaN,NaN,NaN,2024-04-10 14:55:00,2024-04-10 16:21:00,3,SYR,2024-04-10 15:00:00JFK,2024-04-10 16:00:00SYR
4,WN 5285,WN,WN,MCO,NaN,NaN,NaN,NaN,2024-04-10 11:05:00,2024-04-10 13:45:00,3,SYR,2024-04-10 11:00:00MCO,2024-04-10 14:00:00SYR


,ORGIN_WTH_name,ORGIN_WTH_datetime,ORGIN_WTH_temp,ORGIN_WTH_feelslike,ORGIN_WTH_precip,ORGIN_WTH_precipprob,ORGIN_WTH_preciptype,ORGIN_WTH_snow,ORGIN_WTH_windgust,ORGIN_WTH_windspeed,ORGIN_WTH_winddir,ORGIN_WTH_cloudcover,ORGIN_WTH_visibility,ORGIN_WTH_severerisk,ORGIN_WTH_conditions,ORGIN_WTH_JOIN
0,JFK,2024-04-08 00:00:00,46.30,45.10,0.00,0.00,NaN,0,8.10,3.40,267.00,0.00,9.90,10,Clear,2024-04-08 00:00:00JFK
1,JFK,2024-04-08 01:00:00,46.20,42.80,0.00,0.00,NaN,0,8.10,6.70,262.00,0.00,9.90,10,Clear,2024-04-08 01:00:00JFK
2,JFK,2024-04-08 02:00:00,46.20,43.20,0.00,0.00,NaN,0,8.10,5.90,262.00,0.00,9.90,10,Clear,2024-04-08 02:00:00JFK
3,JFK,2024-04-08 03:00:00,45.20,42.10,0.00,0.00,NaN,0,8.10,5.70,262.00,0.00,9.90,10,Clear,2024-04-08 03:00:00JFK
4,JFK,2024-04-08 04:00:00,45.10,42.20,0.00,0.00,NaN,0,6.90,5.50,285.00,0.00,9.90,10,Clear,2024-04-08 04:00:00JFK


,DEST_WTH_name,DEST_WTH_datetime,DEST_WTH_temp,DEST_WTH_feelslike,DEST_WTH_precip,DEST_WTH_precipprob,DEST_WTH_preciptype,DEST_WTH_snow,DEST_WTH_windgust,DEST_WTH_windspeed,DEST_WTH_winddir,DEST_WTH_cloudcover,DEST_WTH_visibility,DEST_WTH_severerisk,DEST_WTH_conditions,DEST_WTH_JOIN
0,JFK,2024-04-08 00:00:00,46.30,45.10,0.00,0.00,NaN,0,8.10,3.40,267.00,0.00,9.90,10,Clear,2024-04-08 00:00:00JFK
1,JFK,2024-04-08 01:00:00,46.20,42.80,0.00,0.00,NaN,0,8.10,6.70,262.00,0.00,9.90,10,Clear,2024-04-08 01:00:00JFK
2,JFK,2024-04-08 02:00:00,46.20,43.20,0.00,0.00,NaN,0,8.10,5.90,262.00,0.00,9.90,10,Clear,2024-04-08 02:00:00JFK
3,JFK,2024-04-08 03:00:00,45.20,42.10,0.00,0.00,NaN,0,8.10,5.70,262.00,0.00,9.90,10,Clear,2024-04-08 03:00:00JFK
4,JFK,2024-04-08 04:00:00,45.10,42.20,0.00,0.00,NaN,0,6.90,5.50,285.00,0.00,9.90,10,Clear,2024-04-08 04:00:00JFK


In [59]:
# Join data set
merged_flight_weather = pd.merge(initial_predictions_dataset, org_weather_data, on='ORGIN_WTH_JOIN')
merged_flight_weather = pd.merge(merged_flight_weather, dst_weather_data, on='DEST_WTH_JOIN')

merged_flight_weather.head()
merged_flight_weather.shape
merged_flight_weather.columns

,FLIGHT NUMBER,MKT_UNIQUE_CARRIER,OP_UNIQUE_CARRIER,ORIGIN,ARRIVAL STATUS,ARRIVAL STATUS_Prev_flight_early,ARRIVAL STATUS_Prev_flight_ontime,ARRIVAL STATUS_Prev_flight_late,SCH_DEP_TIME,SCH_ARR_TIME,DAY_OF_WEEK,DEST,ORGIN_WTH_JOIN,DEST_WTH_JOIN,ORGIN_WTH_name,ORGIN_WTH_datetime,ORGIN_WTH_temp,ORGIN_WTH_feelslike,ORGIN_WTH_precip,ORGIN_WTH_precipprob,ORGIN_WTH_preciptype,ORGIN_WTH_snow,ORGIN_WTH_windgust,ORGIN_WTH_windspeed,ORGIN_WTH_winddir,ORGIN_WTH_cloudcover,ORGIN_WTH_visibility,ORGIN_WTH_severerisk,ORGIN_WTH_conditions,DEST_WTH_name,DEST_WTH_datetime,DEST_WTH_temp,DEST_WTH_feelslike,DEST_WTH_precip,DEST_WTH_precipprob,DEST_WTH_preciptype,DEST_WTH_snow,DEST_WTH_windgust,DEST_WTH_windspeed,DEST_WTH_winddir,DEST_WTH_cloudcover,DEST_WTH_visibility,DEST_WTH_severerisk,DEST_WTH_conditions
0,UA 1400,UA,UA,ORD,NaN,NaN,NaN,NaN,2024-04-10 18:52:00,2024-04-10 21:47:00,3,SYR,2024-04-10 19:00:00ORD,2024-04-10 22:00:00SYR,ORD,2024-04-10 19:00:00,54.10,54.10,0.00,34.00,NaN,0,12.80,9.20,70.00,80.00,10.10,10,Partially cloudy,SYR,2024-04-10 22:00:00,54.40,54.40,0.00,40.00,NaN,0,9.20,6.30,126.70,80.30,8.20,10,Partially cloudy
1,AA 3402,AA,MQ,ORD,NaN,NaN,NaN,NaN,2024-04-10 19:59:00,2024-04-10 22:52:00,3,SYR,2024-04-10 20:00:00ORD,2024-04-10 23:00:00SYR,ORD,2024-04-10 20:00:00,53.00,53.00,0.00,27.00,NaN,0,12.10,8.70,50.00,81.30,10.10,10,Partially cloudy,SYR,2024-04-10 23:00:00,53.30,53.30,0.00,40.00,NaN,0,8.90,6.30,130.00,82.00,7.30,10,Partially cloudy
2,B6 116,B6,B6,JFK,NaN,NaN,NaN,NaN,2024-04-10 13:33:00,2024-04-10 14:50:00,3,SYR,2024-04-10 14:00:00JFK,2024-04-10 15:00:00SYR,JFK,2024-04-10 14:00:00,54.90,54.90,0.03,44.00,rain,0,11.40,9.20,110.00,90.00,3.70,10,Partially cloudy,SYR,2024-04-10 15:00:00,63.20,63.20,0.00,45.00,NaN,0,13.40,9.80,283.30,70.70,9.80,10,Partially cloudy
3,DL 5182,DL,9E,JFK,NaN,NaN,NaN,NaN,2024-04-10 14:55:00,2024-04-10 16:21:00,3,SYR,2024-04-10 15:00:00JFK,2024-04-10 16:00:00SYR,JFK,2024-04-10 15:00:00,55.30,55.30,0.00,44.00,NaN,0,11.90,9.20,100.00,88.30,3.30,10,Partially cloudy,SYR,2024-04-10 16:00:00,62.70,62.70,0.00,45.00,NaN,0,13.20,9.60,306.70,66.30,9.40,10,Partially cloudy
4,WN 5285,WN,WN,MCO,NaN,NaN,NaN,NaN,2024-04-10 11:05:00,2024-04-10 13:45:00,3,SYR,2024-04-10 11:00:00MCO,2024-04-10 14:00:00SYR,MCO,2024-04-10 11:00:00,79.10,79.10,0.00,3.00,NaN,0,21.90,15.00,150.00,43.00,10.10,10,Partially cloudy,SYR,2024-04-10 14:00:00,63.90,63.90,0.05,45.00,rain,0,13.90,10.30,260.00,75.00,10.10,10,Partially cloudy


(23, 44)

Index(['FLIGHT NUMBER', 'MKT_UNIQUE_CARRIER', 'OP_UNIQUE_CARRIER', 'ORIGIN',
       'ARRIVAL STATUS', 'ARRIVAL STATUS_Prev_flight_early',
       'ARRIVAL STATUS_Prev_flight_ontime', 'ARRIVAL STATUS_Prev_flight_late',
       'SCH_DEP_TIME', 'SCH_ARR_TIME', 'DAY_OF_WEEK', 'DEST', 'ORGIN_WTH_JOIN',
       'DEST_WTH_JOIN', 'ORGIN_WTH_name', 'ORGIN_WTH_datetime',
       'ORGIN_WTH_temp', 'ORGIN_WTH_feelslike', 'ORGIN_WTH_precip',
       'ORGIN_WTH_precipprob', 'ORGIN_WTH_preciptype', 'ORGIN_WTH_snow',
       'ORGIN_WTH_windgust', 'ORGIN_WTH_windspeed', 'ORGIN_WTH_winddir',
       'ORGIN_WTH_cloudcover', 'ORGIN_WTH_visibility', 'ORGIN_WTH_severerisk',
       'ORGIN_WTH_conditions', 'DEST_WTH_name', 'DEST_WTH_datetime',
       'DEST_WTH_temp', 'DEST_WTH_feelslike', 'DEST_WTH_precip',
       'DEST_WTH_precipprob', 'DEST_WTH_preciptype', 'DEST_WTH_snow',
       'DEST_WTH_windgust', 'DEST_WTH_windspeed', 'DEST_WTH_winddir',
       'DEST_WTH_cloudcover', 'DEST_WTH_visibility', 'DEST_WTH_severeris

In [60]:
# Drop unnecessary columns
merged_flight_weather_updated = merged_flight_weather.drop(columns=['ORGIN_WTH_JOIN', 'DEST_WTH_JOIN', 'ORGIN_WTH_datetime', 'DEST_WTH_datetime', 'ORGIN_WTH_name', 'DEST_WTH_name'])

In [61]:
merged_flight_weather_updated.head()

,FLIGHT NUMBER,MKT_UNIQUE_CARRIER,OP_UNIQUE_CARRIER,ORIGIN,ARRIVAL STATUS,ARRIVAL STATUS_Prev_flight_early,ARRIVAL STATUS_Prev_flight_ontime,ARRIVAL STATUS_Prev_flight_late,SCH_DEP_TIME,SCH_ARR_TIME,DAY_OF_WEEK,DEST,ORGIN_WTH_temp,ORGIN_WTH_feelslike,ORGIN_WTH_precip,ORGIN_WTH_precipprob,ORGIN_WTH_preciptype,ORGIN_WTH_snow,ORGIN_WTH_windgust,ORGIN_WTH_windspeed,ORGIN_WTH_winddir,ORGIN_WTH_cloudcover,ORGIN_WTH_visibility,ORGIN_WTH_severerisk,ORGIN_WTH_conditions,DEST_WTH_temp,DEST_WTH_feelslike,DEST_WTH_precip,DEST_WTH_precipprob,DEST_WTH_preciptype,DEST_WTH_snow,DEST_WTH_windgust,DEST_WTH_windspeed,DEST_WTH_winddir,DEST_WTH_cloudcover,DEST_WTH_visibility,DEST_WTH_severerisk,DEST_WTH_conditions
0,UA 1400,UA,UA,ORD,NaN,NaN,NaN,NaN,2024-04-10 18:52:00,2024-04-10 21:47:00,3,SYR,54.10,54.10,0.00,34.00,NaN,0,12.80,9.20,70.00,80.00,10.10,10,Partially cloudy,54.40,54.40,0.00,40.00,NaN,0,9.20,6.30,126.70,80.30,8.20,10,Partially cloudy
1,AA 3402,AA,MQ,ORD,NaN,NaN,NaN,NaN,2024-04-10 19:59:00,2024-04-10 22:52:00,3,SYR,53.00,53.00,0.00,27.00,NaN,0,12.10,8.70,50.00,81.30,10.10,10,Partially cloudy,53.30,53.30,0.00,40.00,NaN,0,8.90,6.30,130.00,82.00,7.30,10,Partially cloudy
2,B6 116,B6,B6,JFK,NaN,NaN,NaN,NaN,2024-04-10 13:33:00,2024-04-10 14:50:00,3,SYR,54.90,54.90,0.03,44.00,rain,0,11.40,9.20,110.00,90.00,3.70,10,Partially cloudy,63.20,63.20,0.00,45.00,NaN,0,13.40,9.80,283.30,70.70,9.80,10,Partially cloudy
3,DL 5182,DL,9E,JFK,NaN,NaN,NaN,NaN,2024-04-10 14:55:00,2024-04-10 16:21:00,3,SYR,55.30,55.30,0.00,44.00,NaN,0,11.90,9.20,100.00,88.30,3.30,10,Partially cloudy,62.70,62.70,0.00,45.00,NaN,0,13.20,9.60,306.70,66.30,9.40,10,Partially cloudy
4,WN 5285,WN,WN,MCO,NaN,NaN,NaN,NaN,2024-04-10 11:05:00,2024-04-10 13:45:00,3,SYR,79.10,79.10,0.00,3.00,NaN,0,21.90,15.00,150.00,43.00,10.10,10,Partially cloudy,63.90,63.90,0.05,45.00,rain,0,13.90,10.30,260.00,75.00,10.10,10,Partially cloudy


In [62]:
merged_flight_weather_updated.columns

Index(['FLIGHT NUMBER', 'MKT_UNIQUE_CARRIER', 'OP_UNIQUE_CARRIER', 'ORIGIN',
       'ARRIVAL STATUS', 'ARRIVAL STATUS_Prev_flight_early',
       'ARRIVAL STATUS_Prev_flight_ontime', 'ARRIVAL STATUS_Prev_flight_late',
       'SCH_DEP_TIME', 'SCH_ARR_TIME', 'DAY_OF_WEEK', 'DEST', 'ORGIN_WTH_temp',
       'ORGIN_WTH_feelslike', 'ORGIN_WTH_precip', 'ORGIN_WTH_precipprob',
       'ORGIN_WTH_preciptype', 'ORGIN_WTH_snow', 'ORGIN_WTH_windgust',
       'ORGIN_WTH_windspeed', 'ORGIN_WTH_winddir', 'ORGIN_WTH_cloudcover',
       'ORGIN_WTH_visibility', 'ORGIN_WTH_severerisk', 'ORGIN_WTH_conditions',
       'DEST_WTH_temp', 'DEST_WTH_feelslike', 'DEST_WTH_precip',
       'DEST_WTH_precipprob', 'DEST_WTH_preciptype', 'DEST_WTH_snow',
       'DEST_WTH_windgust', 'DEST_WTH_windspeed', 'DEST_WTH_winddir',
       'DEST_WTH_cloudcover', 'DEST_WTH_visibility', 'DEST_WTH_severerisk',
       'DEST_WTH_conditions'],
      dtype='object')

In [63]:
columns_to_drop = ['FLIGHT NUMBER', 'ARRIVAL STATUS', 'ARRIVAL STATUS_Prev_flight_early', 'ARRIVAL STATUS_Prev_flight_ontime', 'ARRIVAL STATUS_Prev_flight_late', 'ORGIN_WTH_feelslike',
                    'ORGIN_WTH_windgust', 'DEST_WTH_feelslike', 'DEST_WTH_windgust',
                    'DEST', 'DEST_WTH_preciptype', 'ORGIN_WTH_preciptype',
                    'DEST_WTH_conditions', 'ORGIN_WTH_conditions']
flight_weather_data_updated =  merged_flight_weather_updated.drop(columns = columns_to_drop)
flight_weather_data_updated.shape
flight_weather_data_updated.columns

(23, 24)

Index(['MKT_UNIQUE_CARRIER', 'OP_UNIQUE_CARRIER', 'ORIGIN', 'SCH_DEP_TIME',
       'SCH_ARR_TIME', 'DAY_OF_WEEK', 'ORGIN_WTH_temp', 'ORGIN_WTH_precip',
       'ORGIN_WTH_precipprob', 'ORGIN_WTH_snow', 'ORGIN_WTH_windspeed',
       'ORGIN_WTH_winddir', 'ORGIN_WTH_cloudcover', 'ORGIN_WTH_visibility',
       'ORGIN_WTH_severerisk', 'DEST_WTH_temp', 'DEST_WTH_precip',
       'DEST_WTH_precipprob', 'DEST_WTH_snow', 'DEST_WTH_windspeed',
       'DEST_WTH_winddir', 'DEST_WTH_cloudcover', 'DEST_WTH_visibility',
       'DEST_WTH_severerisk'],
      dtype='object')

In [65]:
flight_weather_data_updated.to_csv('./test_data/initial_test_data_for_prediction.csv', index=False)